# Solution to the "Impossible Puzzle"

As it appeared in [The Riddler](http://fivethirtyeight.com/features/can-you-solve-the-impossible-puzzle/)

The key to solving this kind of problem is noticing that the "No, I don't know" contains information about the number Peter and Susan has on their slip of paper. Some products and sums are only possible to arrive at given a specific pair of numbers.

In this case, the fact that Pete doesn't immediately know the numbers tells us that the only possible pairs are
 (1, 4),
 (1, 6),
 (1, 8),
 (1, 9),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 6),
 (2, 8),
 (2, 9),
 (3, 3),
 (3, 4),
 (3, 6), 
 (3, 8),
 (4, 4),
 (4, 6),
 (4, 9), 
 and (6, 6), because any other numbers would have produced a unique product and Pete would have been able to figure out what the two numbers were. The question is somewhat ambiguous as to whether the two numbers are allowed to be the same. It turns out that we have to allow that for the problem to work, but more about that later.

When Susan is asked if she knows the answer, she only has to consider the pairs of numbers mentioned above. In that set, (2, 2), (6, 6), and (4, 9) have unique sums and can be ruled out since Susan answered "no". We can apply this method repeatedly and we should hopefully arrive at a unique solution in the end.

The method described above is easy, but tideous, to perform with pen and paper. A programmatic implementation of it would allow us to study variations of the problem:

- What if we allow more numbers or fewer numbers?
- What if the sequence of is altered? For example, what if Susan got it on her second go?

I decided to subclass the Python `list` for this purpose. I could have written procedural code, but the subclass gave me a nice syntax for chaining method calls. The list keeps track of all pairs of numbers that an observer would regard as possible. Pete and Susan on the other hand have more information.

I created two convenient static methods for inializing the class based on a range of numbers.

The methods `does_pete_know` and `does_susan_know` return a new instance of the class with (potentially) fewer entries. If the `answer` parameter is not set to `'yes'`, all pairs with a unique product or sum will be discarded and vice versa.

In [1]:
class ImpossiblePuzzle(list):
    @staticmethod
    def from_set(s, allow_equal=True):
        """
        Initialize an ImpossiblePuzzle object containing all unordered pairs
        (a, b) where a and b are drawn from the set s.
        """
        s = set(s)
        if allow_equal:
            return ImpossiblePuzzle(
                (i, j) for i in s for j in s if i <= j)
        else:
            return ImpossiblePuzzle(
                (i, j) for i in s for j in s if i < j)

    @staticmethod
    def from_range(*args, **kwargs):
        """
        Initialize an ImpossiblePuzzle object containing all unordered pairs
        (a, b) where a and b are drawn from range(*args).
        """
        allow_equal = kwargs.get('allow_equal', True)
        return ImpossiblePuzzle.from_set(range(*args), allow_equal)

    def does_pete_know(self, answer):
        """
        Return a new ImpossiblePuzzle object based on this one but with entries removed
        based on Pete's answer to the question 'Do you know what the numbers are?'
        """
        p = dict()
        for i, j in self:
            if i*j in p:
                p[i*j].append((i, j))
            else:
                p[i*j] = [(i, j)]
        ret = []
        for pairs in p.values():
            if (len(pairs) > 1) != (answer == 'yes'):
                ret += pairs
        return ImpossiblePuzzle(ret)

    def does_susan_know(self, answer):
        """
        Return a new ImpossiblePuzzle object based on this one but with entries removed
        based on Susan's answer to the question 'Do you know what the numbers are?'
        """
        s = dict()
        for i, j in self:
            if i+j in s:
                s[i+j].append((i, j))
            else:
                s[i+j] = [(i, j)]
        ret = []
        for pairs in s.values():
            if (len(pairs) > 1) != (answer == 'yes'):
                ret += pairs
        return ImpossiblePuzzle(ret)

We instantiate a new puzzle as follows

In [2]:
puzzle = ImpossiblePuzzle.from_range(1, 10)

Now we can use the chaining of method calls to build a sequence of answers from the two participants. Consider my example from above where Susan figures it out on her second go:

In [3]:
(puzzle
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('yes')
)

[(2, 3)]

This means that the only possible pair of numbers is (2, 3).

Another interesting thing is that just because one of the participants has figured it out, the other one may still be uncertain. Consider the case where Susan figures it out the first time she is asked. In this case, Pete has the number 36 and Susan has either 12 or 13.

In [4]:
(puzzle
  .does_pete_know('no')
  .does_susan_know('yes')
  .does_pete_know('no')
)

[(6, 6), (4, 9)]

Notice also that if Pete does figure it out, an observer has enough information to deduce the numbers.

In [5]:
(puzzle
  .does_pete_know('no')
  .does_susan_know('yes')
  .does_pete_know('yes')
)

[(2, 2)]

## Can the numbers be equal

As promised, let's investigate what happens if the numbers are not allowed to be equal. The static methods take the optional parameter `allow_equal` which defaults to `True`. We can set this to `False` and see what happens.

In [6]:
puzzle2 = ImpossiblePuzzle.from_range(1, 10, allow_equal=False)

In [7]:
(puzzle2
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
 )

[(3, 6), (2, 9)]

It should be clear that whether Susan has 9 or 11 she will know the answer at this point. Pete on the other hand will never know the answer.

## What if we allow more numbers?

Instead of the original puzzle where numbers were from the range 1 to 9, what if we allowed numbers from 1 to 99? This would be very timeconsuming to solve manually, but the computer can do it fast and accurate. In this case, I use the same sequence of answers as in the original example.

In [8]:
puzzle99 = ImpossiblePuzzle.from_range(1, 100)

(puzzle99
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('yes')
)

[(77, 90)]

The original puzzle had a follow up question. What if Pete answered "no" the fifth time, would Susan be able to solve it? In our extended 99 number version the answer is yes, but only if the numbers are (72, 95).

In [9]:
(puzzle99
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
 .does_susan_know('yes')
)

[(72, 95)]

## Solutions to the original questions

Finally, I put the code for solving the original questions, but to avoid spoiling your fun I leave those cells unevaluated.

First up Pete figures it out on the fifth go, what are the two numbers?

In [ ]:
(puzzle
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('yes')
)

Second, what if Pete answered "no" the fifth time, would Susan be able to solve it?

In [ ]:
(puzzle
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('yes')
)

If this statement evaluates to an empty list, the answer is no, if it evaluates to a non-empty list, the answer is either "yes", or "yes, if the numbers are one of these pairs".

To figure out which of the two latter is correct, the following statement can be evaluated

In [ ]:
(puzzle
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
  .does_pete_know('no')
  .does_susan_know('no')
)

If this statement evaluates to an empty list and the previous statement evaluated to a non-empty list, the answer is "yes". If both lists are non-empty, the answer is "yes, but..."